In [88]:
#
#
# RefCOCOg Adv Data Visualization
#
#

%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from IPython.display import display, clear_output
import ipywidgets
from random import shuffle
import ipywidgets as widgets
import scipy
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
import matplotlib.patches as mpatches
import os.path as osp
import pickle
import json
import random

In [89]:
# set number of random samples to visualize
RANDOM_SAMPLES = 10

#set the folder where you extracted the Refcocog Adv dataset
#dataset folder structure is as follows:

# DATA_ROOT_FOLDER
#     |
#     ------> Images
#               |
#               ------> *.jpg
#     ------> refcocog_adv_annotations.json

DATA_ROOT_FOLDER = './'

refcocog_adv_annotations = json.load(open(os.path.join(DATA_ROOT_FOLDER, 'refcocog_adv_annotations.json'), 'r'))

ref_adv = {}
ref_easy = {}
ref_hard = {}

In [90]:
refcocog_adv_annotations

{u'36BTXXLZ2VF94FLCT0UIDIPPPACR4P_batch19': {u'GT_bbox_number': u'2',
  u'adj': u'laptop',
  u'adj_noun': u'phone keyboard laptop',
  u'annID': u'327237',
  u'assigns': [{u'dropdownselect': u'1'},
   {u'dropdownselect': u'2'},
   {u'dropdownselect': u'1'}],
  u'batch_id': u'batch19',
  u'bbox_candidates_map': u'{"1": [193.67, 136.99, 292.82, 195.21], "2": [134.19, 127.75, 169.47, 197.07], "3": [16.83, 133.64, 469.77, 195.86]}',
  u'bbox_html_dropdown': u"<option value='0'>Select the Box here</option> <option value='1'>1</option> <option value='2'>2</option> <option value='3'>3</option> ",
  u'hit_id': u'36BTXXLZ2VF94FLCT0UIDIPPPACR4P',
  u'hit_type_id': u'3QH037L2CMDD4RRYVNJWX0CT0B0AWC',
  u'imgID': u'553126',
  u'img_file_name': u'COCO_train2014_000000553126.jpg',
  u'most_confused_bbox_GT_desc': u"the Nokia phone sitting atop another phone that rests upon a laptop's keyboard",
  u'most_confused_bbox_number': u'1',
  u'must_words': u'nokia,phone,laptop,keyboard',
  u'noun': u'phone ke

In [91]:
# Function to draw bounding boxes
def draw_bbox(ax, bbox,edge_color='red',  line_width=4, alpha=0.5, facecolor='red', text=""):
    """Draw one bounding box on a matplotlib axis object (ax)."""
    import matplotlib.patches as mpatches
    import matplotlib.pyplot as plt
    
    bbox_plot = mpatches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3],
        fill=False, edgecolor=facecolor, linewidth=line_width)
    ax.add_patch(bbox_plot)
    ax.text(bbox[0], bbox[1], text, color='white',verticalalignment='top',fontweight='bold',fontsize=18,bbox=dict(edgecolor='none', facecolor=facecolor, alpha=0.5))

In [92]:
ref_adv['annotations'] = []
for ky,ann in refcocog_adv_annotations.items():
    
    new_ann = {}
    new_ann['img_file_name'] = ann['img_file_name']
    new_ann['annID'] = ann['annID']
    new_ann['bboxes'] = ann['bbox_candidates_map']
    new_ann['gt_bbox'] = ann['most_confused_bbox_number']
    new_ann['expression'] = ann['most_confused_bbox_GT_desc']
    new_ann['label'] = 'adv'
    
    ref_adv['annotations'].append(new_ann)

In [93]:
import copy
all_annotations = []
refs = pickle.load(open('./refs(original).p', 'rb'))
test_refs = []

for ref_o in refs:
    ref = copy.deepcopy(ref_o)
    if(ref['split'] == 'test'):
        test_refs.append(ref)

In [94]:
test_refs[0]

{'ann_id': 491042,
 'category_id': 1,
 'file_name': u'COCO_train2014_000000380440_491042.jpg',
 'image_id': 380440,
 'ref_id': 0,
 'sent_ids': [8, 9],
 'sentences': [{'raw': u'the man in yellow coat',
   'sent': u'the man in yellow coat',
   'sent_id': 8,
   'tokens': [u'the', u'man', u'in', u'yellow', u'coat']},
  {'raw': u'Skiier in red pants.',
   'sent': u'skiier in red pants',
   'sent_id': 9,
   'tokens': [u'skiier', u'in', u'red', u'pants']}],
 'split': 'test'}

In [95]:
# hard sent ids
easy_refs = []
hard_refs = []

for ref in test_refs:
    sents = ref['sentences']
    image_id = ref['image_id']
    ann_id = ref['ann_id']
    for sent in sents:
        sent_id = sent['sent_id']
        text = sent['raw']
        tokens = sent['tokens']
        found_flag = 0
        for ky,annot in refcocog_adv_annotations.items():
            if(annot['raw_str'].strip().lower() == text.strip().lower()):
                found_flag = 1
                break;
        if(found_flag == 0):
            new_ann = {}
            new_ann['img_file_name'] = ref['file_name']
            new_ann['annID'] = ann_id
            ann_coco = refexp_val.loadAnns(ann_id)[0]
            
            coco_anns = refexp_val.getAnnIds(imgIds=[image_id])
            bbox_cands = refexp_val.coco.loadAnns(ids=coco_anns)
            
            new_ann['bboxes'] = {}
            cnt = 1
            tmp_box = []
            for cand in bbox_cands:
                new_ann['bboxes'][cnt] = cand['bbox']
                tmp_box.append(cand['bbox'])
                cnt += 1
            new_ann['gt_bbox'] = tmp_box.index(ann_coco['bbox']) + 1
            new_ann['expression'] = text
            new_ann['label'] = 'easy'
            ref_adv['annotations'].append(new_ann)
        else:
            new_ann = {}
            new_ann['img_file_name'] = ref['file_name']
            new_ann['annID'] = ann_id
            ann_coco = refexp_val.loadAnns(ann_id)[0]
            
            coco_anns = refexp_val.getAnnIds(imgIds=[image_id])
            bbox_cands = refexp_val.coco.loadAnns(ids=coco_anns)
            
            new_ann['bboxes'] = {}
            cnt = 1
            tmp_box = []
            for cand in bbox_cands:
                new_ann['bboxes'][cnt] = cand['bbox']
                tmp_box.append(cand['bbox'])
                cnt += 1
            new_ann['gt_bbox'] = tmp_box.index(ann_coco['bbox']) + 1
            new_ann['expression'] = text
            new_ann['label'] = 'hard'
            ref_adv['annotations'].append(new_ann)

In [96]:
len(ref_adv['annotations'])

13306

In [2]:
#%matplotlib inline
import matplotlib
matplotlib.use('Agg')
import os
import sys

In [3]:
sys.path.append('./Google_Refexp_toolbox/google_refexp_py_lib/')
sys.path.append('./Google_Refexp_toolbox/external/coco/PythonAPI/')
sys.path.append('./Google_Refexp_toolbox/coco/PythonAPI/pycocotools/')
from refexp import Refexp

In [4]:
coco_filename       = './data/instances_train2014.json'
#refexp_filename_val = '../data/google_refexp_val_201511_coco_aligned.json'
refexp_filename_val = './data/google_refexp_val_201511_coco_aligned_mcg_umd.json'
refexp_val = Refexp(refexp_filename_val, coco_filename)

loading annotations into memory...
Done (t=13.51s)
creating index...
index created!
Dataset loaded.


In [99]:
with open("adv_easy_hard_annotations.json", "w") as outfile: 
    json.dump(ref_adv, outfile, indent=2)

In [97]:
cnt = 0
for i in ref_adv['annotations']:
    if i['label'] == 'adv':
        cnt += 1
print(cnt)

for i in ref_adv['annotations']:
    if i['label'] == 'easy':
        print(i)

3704
{'annID': 491042, 'label': 'easy', 'gt_bbox': 2, 'bboxes': {1: [228.98, 41.4, 141.95, 270.38], 2: [374.31, 65.06, 136.04, 201.94], 3: [244.21, 251.3, 141.07, 90.63], 4: [291.38, 190.07, 185.19, 68.27]}, 'expression': u'the man in yellow coat', 'img_file_name': u'COCO_train2014_000000380440_491042.jpg'}
{'annID': 491042, 'label': 'easy', 'gt_bbox': 2, 'bboxes': {1: [228.98, 41.4, 141.95, 270.38], 2: [374.31, 65.06, 136.04, 201.94], 3: [244.21, 251.3, 141.07, 90.63], 4: [291.38, 190.07, 185.19, 68.27]}, 'expression': u'Skiier in red pants.', 'img_file_name': u'COCO_train2014_000000380440_491042.jpg'}
{'annID': 398406, 'label': 'easy', 'gt_bbox': 1, 'bboxes': {1: [93.95, 83.29, 504.61, 290.57]}, 'expression': u'There is red colored truck in between the other trucks', 'img_file_name': u'COCO_train2014_000000419645_398406.jpg'}
{'annID': 398406, 'label': 'easy', 'gt_bbox': 1, 'bboxes': {1: [93.95, 83.29, 504.61, 290.57]}, 'expression': u'A shiny red vintage pickup truck', 'img_file_nam

{'annID': 207284, 'label': 'easy', 'gt_bbox': 1, 'bboxes': {1: [38.83, 102.74, 159.78, 165.75], 2: [1.51, 114.02, 55.88, 154.04]}, 'expression': u'a black man in a grey shirt watching tv', 'img_file_name': u'COCO_train2014_000000414130_207284.jpg'}
{'annID': 207284, 'label': 'easy', 'gt_bbox': 1, 'bboxes': {1: [38.83, 102.74, 159.78, 165.75], 2: [1.51, 114.02, 55.88, 154.04]}, 'expression': u'man in grey shirt', 'img_file_name': u'COCO_train2014_000000414130_207284.jpg'}
{'annID': 226457, 'label': 'easy', 'gt_bbox': 1, 'bboxes': {1: [14.29, 52.59, 154.28, 422.86], 2: [298.96, 32.17, 187.89, 516.69], 3: [40.51, 121.98, 282.16, 243.41]}, 'expression': u'the man wearing headphones', 'img_file_name': u'COCO_train2014_000000463467_226457.jpg'}
{'annID': 226457, 'label': 'easy', 'gt_bbox': 1, 'bboxes': {1: [14.29, 52.59, 154.28, 422.86], 2: [298.96, 32.17, 187.89, 516.69], 3: [40.51, 121.98, 282.16, 243.41]}, 'expression': u'Man wearing blue sweater listening to music.', 'img_file_name': u'C

{'annID': 505524, 'label': 'easy', 'gt_bbox': 3, 'bboxes': {1: [63.03, 95.71, 235.8, 361.98], 2: [0.0, 140.42, 296.79, 490.56], 3: [173.36, 116.66, 179.15, 368.4]}, 'expression': u'A woman wearing a long sleeve sweater cutting into a desert.', 'img_file_name': u'COCO_train2014_000000087569_505524.jpg'}
{'annID': 582808, 'label': 'easy', 'gt_bbox': 1, 'bboxes': {1: [108.89, 243.28, 257.76, 234.33]}, 'expression': u'A elephant bigger than the surrounding elephants.', 'img_file_name': u'COCO_train2014_000000499738_582808.jpg'}
{'annID': 621590, 'label': 'easy', 'gt_bbox': 1, 'bboxes': {1: [96.04, 0.27, 165.64, 372.35]}, 'expression': u'Red kite', 'img_file_name': u'COCO_train2014_000000421677_621590.jpg'}
{'annID': 621590, 'label': 'easy', 'gt_bbox': 1, 'bboxes': {1: [96.04, 0.27, 165.64, 372.35]}, 'expression': u'A red, grey, white and black kite.', 'img_file_name': u'COCO_train2014_000000421677_621590.jpg'}
{'annID': 116267, 'label': 'easy', 'gt_bbox': 2, 'bboxes': {1: [78.41, 178.35, 1